In [22]:
# to ensure that the logging statements are shown in juypter output, run this cell
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [23]:
import pandas as pd
# ensure that all columns are shown and that colum content is not cut
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width',1000)

In [24]:
from secfsdstools.update import update

update()

2024-08-07 06:34:30,159 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2024-08-07 06:34:30,173 [INFO] updateprocess  Check if new report zip files are available...
2024-08-07 06:34:30,228 [INFO] updateprocess  check if there are new files to download from sec.gov ...
2024-08-07 06:34:30,925 [INFO] updateprocess  start to transform to parquet format ...
2024-08-07 06:34:30,938 [INFO] updateprocess  start to index parquet files ...


No rapid-api-key is set: 
If you are interested in daily updates, please have a look at https://rapidapi.com/hansjoerg.wingeier/api/daily-sec-financial-statement-dataset


In [25]:
from secfsdstools.e_collector.reportcollecting import SingleReportCollector
from secfsdstools.e_filter.rawfiltering import ReportPeriodAndPreviousPeriodRawFilter
from secfsdstools.e_presenter.presenting import StandardStatementPresenter

# the unique identifier for apple's 10-Q Q2 report of 2024
apple_10q_q2_2024_adsh = "0000320193-24-000069"

# us a Collector to grab the data of the 10-K report. filter for balancesheet information
collector: SingleReportCollector = SingleReportCollector.get_report_by_adsh(
      adsh=apple_10q_q2_2024_adsh
)  
rawdatabag = collector.collect() # load the data from the disk

2024-08-07 06:34:31,028 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg


In [26]:
sub_df = rawdatabag.sub_df
num_df = rawdatabag.num_df
pre_df = rawdatabag.pre_df

In [27]:
sub_df = sub_df[sub_df.adsh=="0000320193-24-000069"]
num_df = num_df[num_df.adsh=="0000320193-24-000069"]
pre_df = pre_df[pre_df.adsh=="0000320193-24-000069"]

In [28]:
print(sub_df.shape)
print(num_df.shape)
print(pre_df.shape)

(1, 36)
(271, 9)
(171, 10)


In [29]:
num_df.coreg = None

In [30]:
num_df = num_df[num_df.coreg.isna()]

In [31]:
num_df.uom.value_counts()

USD       251
shares     20
Name: uom, dtype: int64

In [32]:
# we want to keep all values with uoms that are  not in upper case
mask_has_lower = ~num_df.uom.str.isupper()

# currency has always 3 letters, so we want to keep everything that has a different length
mask_is_none_currency = num_df.uom.str.len() != 3

# keep USD
mask_usd_only = num_df.uom == "USD"

num_df = num_df[mask_has_lower | mask_is_none_currency | mask_usd_only]

In [33]:
print(num_df.shape)

(271, 9)


In [34]:
# get the value of the "period" column for the entry in sub_df
# (there is only one entry left, since filtered for a certain adsh)
period = sub_df.iloc[0].period

# mask the datapoints for the current period
mask_current = num_df.ddate == period

# since period and ddate are actually numbers in the form of YYYYMMDD
# we can simply subtract 10'000 to get the previous year
# !!!! ATTENTION !!! 
# When the period is end of February, we have to consider the leap years!
mask_previous = num_df.ddate == (period - 10000)

num_df = num_df[mask_current | mask_previous]

In [35]:
print(num_df.shape)

(222, 9)


In [36]:
num_df = num_df[num_df.qtrs.isin([1,2])]

In [50]:
num_df[num_df.tag=="RevenueFromContractWithCustomerExcludingAssessedTax"]

,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote
241,0000320193-24-000069,RevenueFromContractWithCustomerExcludingAssessedTax,us-gaap/2023,None,20240331,1,USD,9.075300e+10,None
242,0000320193-24-000069,RevenueFromContractWithCustomerExcludingAssessedTax,us-gaap/2023,None,20230331,1,USD,9.483600e+10,None
243,0000320193-24-000069,RevenueFromContractWithCustomerExcludingAssessedTax,us-gaap/2023,None,20240331,2,USD,2.103280e+11,None
244,0000320193-24-000069,RevenueFromContractWithCustomerExcludingAssessedTax,us-gaap/2023,None,20230331,2,USD,2.119900e+11,None


In [37]:
print(num_df.shape)

(172, 9)


In [38]:
pre_df = pre_df[pre_df.stmt == 'IS']

In [39]:
print(pre_df.shape)

(24, 10)


In [40]:
pre_num_df = pd.merge(num_df,
                      pre_df,
                      on=['adsh', 'tag', 'version'])

In [41]:
print(pre_num_df.shape)

(60, 16)


In [42]:
# mask the entries with the negating flag set and inverse the value column
pre_num_df.loc[pre_num_df.negating == 1, 'value'] = -pre_num_df.value

In [43]:
pre_num_df = pre_num_df[['tag', 'line', 'report', 'uom', 'value', 'ddate', 'qtrs']]

In [46]:
pivot_df = pre_num_df.pivot_table(
                index=['tag','report', 'line', 'uom'],
                columns=['qtrs', 'ddate'],
                values='value')

In [47]:
sort_df = pivot_df.sort_values(['report', 'line'])

In [48]:
sort_df

qtrs                                                                                                                       1                           2              
ddate                                                                                                               20230331      20240331      20230331      20240331
tag                                                                                         report line uom                                                           
RevenueFromContractWithCustomerExcludingAssessedTax                                         2      7    USD     9.483600e+10  9.075300e+10  2.119900e+11  2.103280e+11
CostOfGoodsAndServicesSold                                                                  2      8    USD     5.286000e+10  4.848200e+10  1.196820e+11  1.132020e+11
GrossProfit                                                                                 2      9    USD     4.197600e+10  4.227100e+10  9.230800e+10  9.712600e+10
ResearchAndDevelopmentExpense                                                               2      11   USD     7.457000e+09  7.903000e+09  1.516600e+10  1.559900e+10
SellingGeneralAndAdministrativeExpense                                                      2      12   USD     6.201000e+09  6.468000e+09  1.280800e+10  1.325400e+10
OperatingExpenses                                                                           2      13   USD     1.365800e+10  1.437100e+10  2.797400e+10  2.885300e+10
OperatingIncomeLoss                                                                         2      14   USD     2.831800e+10  2.790000e+10  6.433400e+10  6.827300e+10
NonoperatingIncomeExpense                                                                   2      15   USD     6.400000e+07  1.580000e+08 -3.290000e+08  1.080000e+08
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest 2      16   USD     2.838200e+10  2.805800e+10  6.400500e+10  6.838100e+10
IncomeTaxExpenseBenefit                                                                     2      17   USD     4.222000e+09  4.422000e+09  9.847000e+09  1.082900e+10
NetIncomeLoss                                                                               2      18   USD     2.416000e+10  2.363600e+10  5.415800e+10  5.755200e+10
EarningsPerShareBasic                                                                       2      20   USD     1.530000e+00  1.530000e+00  3.420000e+00  3.720000e+00
EarningsPerShareDiluted                                                                     2      21   USD     1.520000e+00  1.530000e+00  3.410000e+00  3.710000e+00
WeightedAverageNumberOfSharesOutstandingBasic                                               2      23   shares  1.578715e+10  1.540586e+10  1.583994e+10  1.545781e+10
WeightedAverageNumberOfDilutedSharesOutstanding                                             2      24   shares  1.584705e+10  1.546471e+10  1.590138e+10  1.552068e+10